## Notebook to run the fitter on multiple ramps

In [ ]:
import numpy as np
from ramp_utils.ramp import RampTimeSeq,RampMeasurement
from ramp_utils.fitter import IterativeFitter
import time, sys, pickle, bz2
from multiprocessing import Pool
from scipy.interpolate import interp1d

%matplotlib notebook

### Set the working directory for saving the results

In [ ]:
dirsave = '/user/gennaro/Functional_work/Up_the_ramp_myfork/Simulations_results/'
filename = 'Test_1.pbz2'

#### These are two auxiliary function used to generate CRhits 
#### A good number for WFC3 can be found in http://www.stsci.edu/hst/wfc3/documents/ISRs/WFC3-2009-40.pdf, with ~ 2.5e-5 hits per second per pixel


In [ ]:
def generateCR_DNs(size):
    '''
    Just generate random uniform DNs
    '''
    return 1000.*np.random.uniform(size=size)

def generateCR(myramp,CRrate=2.5e-5):
    '''
    Main CR generation function that generates CR hit times and depositied counts
    
    :CRrate:
        rate of CR hits in hits/second
    '''
    
    myCRnumber = np.random.poisson(lam=CRrate*myramp.read_times[-1])

    if myCRnumber > 0:
        mytimes = myramp.read_times[-1]*np.random.uniform(size=myCRnumber)
        mycounts = generateCR_DNs(myCRnumber)
        myCRdict = {'times':mytimes.tolist(),'counts':mycounts.tolist()}
    else:
        myCRdict = None

    return myCRdict

#### Auxiliary function to generate cumulative background electrons
Used to test the fitter with non-constant flux

In [ ]:
def get_vbg_electrons(times,vbg_cr,meas,mean_bg_cr=None):
    '''
    Given a tabulated form for the variable background time dependency,
    generate a number of electrons per each read interval in the ramp
    
    :times:
        times at which the countrate is tabulated
    
    :vbg_cr:
        values of the time variable background at those times   
    
    :meas:
        a RampMeasurement object
    
    :mean_bg_cr:
        the mean value for normalizing the countrate within the interval    
    '''

    
    #Create an interpolator from the tabulated values and interpolate at the ramp read times
    bg_int = interp1d(times,vbg_cr,'quadratic')
    varbg = bg_int(meas.RTS.read_times)

    #Normalize if requested
    if mean_bg_cr is not None:
        dt = meas.RTS.read_times[-1]-meas.RTS.read_times[0]
        t_avg = np.trapz(varbg,meas.RTS.read_times) / dt
        varbg = varbg/t_avg * mean_bg_cr

    #Get the total accumulated electrons
    bg_e=[0]
    bg_e.extend([np.random.poisson(lam=vb*dt) for vb,dt in zip(0.5*(varbg[1:]+varbg[:-1]),meas.RTS.read_times[1:]-meas.RTS.read_times[:-1]) ])
    bg_e = np.asarray(bg_e)
    
    return np.cumsum(bg_e)


### Define the read sequences

In [ ]:
dt,nf,ns,ng = 6.,1,0,17
myramp1 = RampTimeSeq('GENERIC',ng,nframes=nf, nskips=ns, read_times=dt*np.arange(ng*(nf+ns)))

dt,nf,ns,ng = 8.,1,0,13
myramp2 = RampTimeSeq('GENERIC',ng,nframes=nf, nskips=ns, read_times=dt*np.arange(ng*(nf+ns)))

dt,nf,ns,ng = 12.,1,0,9
myramp3 = RampTimeSeq('GENERIC',ng,nframes=nf, nskips=ns, read_times=dt*np.arange(ng*(nf+ns)))

dt,nf,ns,ng = 16.,1,0,7
myramp4 = RampTimeSeq('GENERIC',ng,nframes=nf, nskips=ns, read_times=dt*np.arange(ng*(nf+ns)))

dt,nf,ns,ng = 24.,1,0,5
myramp5 = RampTimeSeq('GENERIC',ng,nframes=nf, nskips=ns, read_times=dt*np.arange(ng*(nf+ns)))


myramp6 = RampTimeSeq('HST/WFC3/IR',15,samp_seq='SPARS100') 


for ramp in [myramp6]:
    ramp.test_plot()

### Define the detector charachteristics
This step is necessary when the ramps are of **GENERIC** type

In [ ]:
gain=1
RON=20
KTC=50
bias=10000
full_well=100000

### Setup the properties of the measurements on which to run the fitter

In [ ]:
myfluxes = [1.,4,16,64]
myramps  = [myramp6,myramp6,myramp6,myramp6]
CRrate   = 5e-4

tbg = np.linspace(0,1500,10)
cbg = np.array([1.,1.2,1.5,1.3,1.7,2.,2.2,2.4,2.0,1.5])

mybgs = [None,None,None,None]
#         {'times':tbg,'vbg_cr':cbg,'mean_bg_cr':0.5}
#         {'times':tbg,'vbg_cr':cbg,'mean_bg_cr':1.}
#        ]


### Setup the fitter method and options

In [ ]:
myfitpars = {'one_iteration_method':'Nelder-Mead'}


### Setup the job

In [ ]:
ntest    = 25
printevery = 5
n_jobs = 4

### Run the fitter on multiple ramps

#### Function that does the job for a single flux/ramp/background combination

In [ ]:
def one_flux_one_ramp(j,myflux,myramp,extra_bg=None):

    
    ts = time.time()

    goodints_p = np.empty([ntest,len(myramp.group_times)-1],dtype=np.bool_)
    CRlist_p   = []
    meas_p     = []
    counter_p  = np.empty(ntest)
    error_p    = np.empty(ntest)
    outerate_p = np.empty(ntest)
    crloops_counter_p  = np.empty(ntest)
    gof_stat_p = np.empty(ntest)
    gof_pval_p = np.empty(ntest)
    extra_bg_p = []

    for i in range(ntest):
        myCRdict = generateCR(myramp,CRrate=CRrate)
        CRlist_p.append(myCRdict)
        
        if myramp.detector == 'GENERIC':
            mymeas = RampMeasurement(myramp,myflux,gain=1,RON=RON,KTC=KTC,bias=bias,full_well=full_well,CRdict=myCRdict)
        else:
            mymeas = RampMeasurement(myramp,myflux,CRdict=myCRdict)
        
        if extra_bg is not None:
            ebh = get_vbg_electrons(extra_bg['times'],extra_bg['vbg_cr'],mymeas,mean_bg_cr=extra_bg['mean_bg_cr'])   
            mymeas.add_background(ebh)
        else:
            ebh = None

        meas_p.append(mymeas)
        extra_bg_p.append(ebh)
        
        myfitter = IterativeFitter(mymeas,fitpars = myfitpars)
        error_p[i],counter_p[i], goodints_p[i,:], crloops_counter_p[i]  = myfitter.perform_fit()
        outerate_p[i] = myfitter.mean_electron_rate
        myfitter.goodness_of_fit(mode='Squared-deviations')
        gof_stat_p[i] = myfitter.gof_stat
        gof_pval_p[i] = myfitter.gof_pval
        
            
        if ((i+1) % printevery) == 0:
            print("{0:5.1%}".format(float(i+1) / ntest),'of', j+1, 'out of', len(myfluxes))
            sys.stdout.flush()
            
    te = time.time()
    print('Elapsed time for js {}: {:8.2f} minutes'.format(j,(te-ts)/60.))

    return j,goodints_p,CRlist_p,meas_p,counter_p,error_p,crloops_counter_p,outerate_p,gof_stat_p,gof_pval_p,extra_bg_p




#### Cell that iterates over all the input fluxes/ramp/background combinations

In [ ]:
inputs = [ [j,mytuple[0],mytuple[1],mytuple[2]] for j,mytuple in enumerate(zip(myfluxes,myramps,mybgs))]
print('js   Fluxes  Ngroups    Mean varbg')
for inp in inputs:
    
    if inp[3] is None:
        cbg = 'None'
    else:
        cbg = inp[3]['mean_bg_cr']
    
    print('{:2d} {:8.3f}    {:3d}         {}'.format(inp[0],inp[1],inp[2].ngroups,cbg))

mypool = Pool(n_jobs)
js,goodints,CRlist,meas_list,counter_list,error_list,crloops_counter_list,outerate_list,gof_stat_list,gof_pval_list,extra_bg_list = map(list, zip(*mypool.starmap(one_flux_one_ramp,inputs)))

### Save the results

In [ ]:
dicttosave = {'js':js,
              'goodints':goodints,
              'CRlist':CRlist,
              'meas_list':meas_list,
              'counter_list':counter_list,
              'error_list':error_list,
              'crloops_counter_list':crloops_counter_list,
              'outerate_list':outerate_list,
              'gof_stat_list':gof_stat_list,
              'gof_pval_list':gof_pval_list,
              'myfluxes':myfluxes,
              'myramps':myramps,
              'extra_bg_list':extra_bg_list
             }
              
with bz2.BZ2File(dirsave+filename, 'w') as f:
        pickle.dump(dicttosave, f)

